In [1]:
import os
import pandas as pd
import re

In [132]:
def loadGFA():
    filename = os.path.join("../Inputs/GFADataset.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} not found")
    df_GFA = pd.read_csv(filename, sep=",")
    dfGFA=df_GFA.rename(columns={"Alloy Formula":"Formula","Phase Formation":"Phase"})
    return dfGFA
    #print(dfGFA)

def loadTablaPeriodica():
    filename = os.path.join("../Inputs/TablaPeriodica.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} not found")
    df_Tabla = pd.read_csv(filename, sep=";")
    df_TablaPeriodica=df_Tabla.rename(columns={"Eea (ev)":"Eea(ev)","I1 (ev)":"I1(ev)","I2 (ev)":"I2(ev)","Tm (K)":"Tm(K)","Rm (nm)":"Rm(nm)","Rc (nm)":"Rc(nm)","Cp (J/molK)":"Cp(J/molK)","K (W/m)/K 300K":"K(W/m)/K300K","Hf (kJ/mol)":"Hf(kJ/mol)","Tb (K)":"Tb(K)"})
    return df_TablaPeriodica
    #print(df_Tabla)

def loadErroresModelo():
    module_path = os.path.dirname(__file__)
    filename = os.path.join(module_path, "../Inputs/ErroresModelo.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} not found")
    df_ErroresModelo = pd.read_csv(filename, sep=";")
    return df_ErroresModelo

In [133]:
def formulaWithoutNumber(dfGFA):
    #Definir un df para guardar el resulrado
    dfCompound = pd.DataFrame(columns =['Formula', 'Phase'])
    #Para cada copuesto en el archivo que solo tenga caracteres, se hace la separacion por elementos
    # y se divide 100 entre la cantidad de elementos encontrados
    for index,row in dfGFA.iterrows():
        formulaTx = row['Formula']
        Phase = row['Phase']
        #Seleccionar compuestos sin valores numericos
        if str.isalpha(formulaTx):
            for char in formulaTx:
                if char.isupper():
                    element = re.findall('[A-Z][^A-Z]*', formulaTx)
            cantComp=str(round(100/(len(element)),1))
            lisComponentsCantComponent = [x + cantComp for x in element]
            strComponentsCantComponent = ''.join([str(elem) for elem in lisComponentsCantComponent])
            dfCompound.loc[len(dfCompound)] = strComponentsCantComponent,Phase       
    return(dfCompound)

In [134]:
def CompleteListElements(CantElemen,IndxElements,formulaTx):
    if CantElemen < 8:
        for i in range(CantElemen,8):
            nanStr = ''
            IndxElements.append(nanStr)
    IndxElements.append(str(CantElemen))
    IndxElements.insert(0,str(formulaTx))
    return(IndxElements)

In [135]:
def findIndexPosition(dfGFA,dfElements):
    print('Estoy en la funcion findIndexPosition')
    #Declaracion de listas vacias
    PosElement=[]
    IndxElements=[]
    IndxNextElement=[]
    LengElement=[]

    #variables temporales para ejecuciones pequeñas
    #elements = {'Element':['Ca','Ag','Fe','Cu','B','Cr','Mo','C','Co','Ni','Pd']}
    #formulas = {'Formula':['Ag27Cu73','Cu0.5NiAlCoCrFe','Fe30Cr30Mo15C15B10','B18.5Co81.5','[(Ni0.9Fe0.1)0.75B0.2Si0.05]96Nb4','Co20.9Pd62.6Si16.5','AlCrMoSiTi']}
    #fin de las variables temporales

    #temp_dfGFA= pd.DataFrame(data=formulas)
    #temp_dfElements= pd.DataFrame(data=elements)

    #temp_dfGFA= read_files.loadGFA()
    #dfFormula=temp_dfGFA
    #temp_dfElements = read_files.loadTablaPeriodica() 
    #dfElements=temp_dfElements

    #Procesa las formulas que no tienen valores de proporcion
    dfNotNumbe = formulaWithoutNumber(dfGFA)

    #Unir los dfs
    dfGFA = (pd.concat([dfGFA, dfNotNumbe]))

    #Definir un df para guardar el resulrado
    dfElementsOfCompos = pd.DataFrame(columns =['Formula', 'Elem1','Elem2','Elem3','Elem4','Elem5','Elem6','Elem7','Elem8','CantElemen'])

    for index, row in dfGFA.iterrows():
        for index, row1 in dfElements.iterrows():
            formulaTx = row['Formula']
            elementTx = row1['Element']
            posIndex = formulaTx.find(elementTx)
            lenElement = len(elementTx)
            nextSubStr = formulaTx[posIndex+lenElement:posIndex+lenElement+1]
            if posIndex >= 0 and nextSubStr.isnumeric() and not str.isalpha(formulaTx):
                PosElement.append(posIndex)                
                substr = formulaTx[posIndex:posIndex+lenElement]
                IndxElements.append(substr)
                IndxNextElement.append(nextSubStr)
                LengElement.append(lenElement)
        CantElemen= (len(IndxElements))  
        #llamado a la funcion CompleteListElements   
        completeList=CompleteListElements(CantElemen,IndxElements,formulaTx)
        
        #Obtener un df con todos los elementos presentes en un formula
        dfElementsOfCompos.loc[len(dfElementsOfCompos)] = completeList
        IndxElements=[]
        PosElement=[]
        IndxNextElement=[]
        LengElement=[]
    print('Imprimiendo el dfElementsOfCompos')
    print(dfElementsOfCompos)
    return dfElementsOfCompos

In [136]:
dfGFA = loadGFA()
dfElements = loadTablaPeriodica()
dfGFA.head()

,Formula,Phase
0,Ag27Cu73,CRA
1,Ag35Ca65,BMG
2,Ag42Fe58,CRA
3,Ag48Cu52,CRA
4,Ag48Fe52,CRA


In [152]:
df = findIndexPosition(dfGFA,dfElements)
df.head()

Estoy en la funcion findIndexPosition
Imprimiendo el dfElementsOfCompos
                                              Formula Elem1 Elem2 Elem3 Elem4  \
0                                            Ag27Cu73    Ag    Cu               
1                                            Ag35Ca65    Ag    Ca               
2                                            Ag42Fe58    Ag    Fe               
3                                            Ag48Cu52    Ag    Cu               
4                                            Ag48Fe52    Ag    Fe               
...                                               ...   ...   ...   ...   ...   
6478                   Zr20.0Hf20.0Ti20.0Cu20.0Fe20.0    Cu    Fe    Hf    Ti   
6479                   Zr20.0Hf20.0Ti20.0Cu20.0Ni20.0    Cu    Hf    Ni    Ti   
6480             Al16.7Cr16.7Mo16.7Ta16.7Ti16.7Zr16.7    Al    Cr    Mo    Ta   
6481             Sr16.7Ca16.7Yb16.7Mg16.7Zn16.7Cu16.7    Ca    Cu    Mg    Sr   
6482  Al12.5Mo12.5Nb12.5Si12.5Ta12.5T

,Formula,Elem1,Elem2,Elem3,Elem4,Elem5,Elem6,Elem7,Elem8,CantElemen
0,Ag27Cu73,Ag,Cu,,,,,,,2
1,Ag35Ca65,Ag,Ca,,,,,,,2
2,Ag42Fe58,Ag,Fe,,,,,,,2
3,Ag48Cu52,Ag,Cu,,,,,,,2
4,Ag48Fe52,Ag,Fe,,,,,,,2


In [153]:
#example_compuesto = "{[(Fe0.6Co0.4)0.75B0.2Si0.05]0.96Nb0.04}96Cr4"
def findCompositionPercentage(example_compuesto):
    if example_compuesto.count('(') > 0:
        #Si el compuesto tiene parentesis al inicio y al final, pasar
        if example_compuesto.startswith('(') and example_compuesto.endswith(')'):
            pass
        else:
            match = re.search(r'\((.*?)\)(\d+\.?\d*)', example_compuesto)
            compuesto = match.group(1)
            multiplicador = match.group(2)

            compuesto = re.sub(r'(\d+\.?\d*)', lambda x: str(float(x.group(1))* float(multiplicador)), compuesto)

            #Reemplazar el compuesto en la formula
            example_compuesto = example_compuesto.replace(match.group(0), compuesto)
    if example_compuesto.count('[') > 0:
        match = re.search(r'\[(.*?)\](\d+\.?\d*)', example_compuesto)
        compuesto = match.group(1)
        multiplicador = match.group(2)

        compuesto = re.sub(r'(\d+\.?\d*)', lambda x: str(float(x.group(1))* float(multiplicador)), compuesto)

        #Reemplazar el compuesto en la formula
        example_compuesto = example_compuesto.replace(match.group(0), compuesto)
    if example_compuesto.count('{') > 0:
        match = re.search(r'\{(.*?)\}(\d+\.?\d*)', example_compuesto)
        compuesto = match.group(1)
        multiplicador = match.group(2)

        compuesto = re.sub(r'(\d+\.?\d*)', lambda x: str(float(x.group(1))* float(multiplicador)), compuesto)

        #Reemplazar el compuesto en la formula
        example_compuesto = example_compuesto.replace(match.group(0), compuesto)
    
    #Separar los numeros de la formula y guardarlos en una lista
    number = re.findall(r'\d+\.?\d*', example_compuesto)
    #Guardar en una lista lo restante de la formula
    element = re.split(r'\d+\.?\d*', example_compuesto)
    #Unir las listas en un diccionario
    dict_compuesto = dict(zip(element, number))
    #Convertir el diccionario en un dataframe
    return dict_compuesto


In [143]:
example_compuesto = "{[(Fe0.6Co0.4)0.75B0.2Si0.05]0.96Nb0.04}96Cr4"
findCompositionPercentage(example_compuesto)

{'Fe': '41.471999999999994',
 'Co': '27.648000000000003',
 'B': '18.432000000000002',
 'Si': '4.6080000000000005',
 'Nb': '3.84',
 'Cr': '4'}

In [154]:
df.head()

,Formula,Elem1,Elem2,Elem3,Elem4,Elem5,Elem6,Elem7,Elem8,CantElemen
0,Ag27Cu73,Ag,Cu,,,,,,,2
1,Ag35Ca65,Ag,Ca,,,,,,,2
2,Ag42Fe58,Ag,Fe,,,,,,,2
3,Ag48Cu52,Ag,Cu,,,,,,,2
4,Ag48Fe52,Ag,Fe,,,,,,,2


In [158]:
df_copy = df.copy()
#Agreguemos 8 columnas nuevas al df, con nombre "Composicion1", "Composicion2", etc
df_copy = pd.concat([df_copy, pd.DataFrame(columns=['Composicion1', 'Composicion2', 'Composicion3', 'Composicion4', 'Composicion5', 'Composicion6', 'Composicion7', 'Composicion8'])])
df_copy
#Funcion para agregar los porcentajes de cada elemento en cada fila
def addCompositionPercentage(df):
    for index, row in df.iterrows():
        formula = row['Formula']
        composicion = findCompositionPercentage(formula)
        for i in range(1,9):
            if row[f'Elem{i}'] in composicion:
                df_copy.loc[index, f'Composicion{i}'] = composicion[row[f'Elem{i}']]

    return df_copy
    
addCompositionPercentage(df_copy)
df_copy

,Formula,Elem1,Elem2,Elem3,Elem4,Elem5,Elem6,Elem7,Elem8,CantElemen,Composicion1,Composicion2,Composicion3,Composicion4,Composicion5,Composicion6,Composicion7,Composicion8
0,Ag27Cu73,Ag,Cu,,,,,,,2,27,73,NaN,NaN,NaN,NaN,NaN,NaN
1,Ag35Ca65,Ag,Ca,,,,,,,2,35,65,NaN,NaN,NaN,NaN,NaN,NaN
2,Ag42Fe58,Ag,Fe,,,,,,,2,42,58,NaN,NaN,NaN,NaN,NaN,NaN
3,Ag48Cu52,Ag,Cu,,,,,,,2,48,52,NaN,NaN,NaN,NaN,NaN,NaN
4,Ag48Fe52,Ag,Fe,,,,,,,2,48,52,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6478,Zr20.0Hf20.0Ti20.0Cu20.0Fe20.0,Cu,Fe,Hf,Ti,Zr,,,,5,20.0,20.0,20.0,20.0,20.0,NaN,NaN,NaN
6479,Zr20.0Hf20.0Ti20.0Cu20.0Ni20.0,Cu,Hf,Ni,Ti,Zr,,,,5,20.0,20.0,20.0,20.0,20.0,NaN,NaN,NaN
6480,Al16.7Cr16.7Mo16.7Ta16.7Ti16.7Zr16.7,Al,Cr,Mo,Ta,Ti,Zr,,,6,16.7,16.7,16.7,16.7,16.7,16.7,NaN,NaN
6481,Sr16.7Ca16.7Yb16.7Mg16.7Zn16.7Cu16.7,Ca,Cu,Mg,Sr,Yb,Zn,,,6,16.7,16.7,16.7,16.7,16.7,16.7,NaN,NaN
